# Trends Analyser — Multi-Agent Research Pipeline

A modular **Researcher → Analyst → Writer** workflow built with the OpenAI Agents SDK.  
The Researcher agent is isolated so it can be swapped for a different domain-specific researcher without touching the rest of the pipeline.

## 1. Setup & Dependencies

In [ ]:
!pip install -q openai-agents python-dotenv pydantic requests

## 2. Imports & Environment

In [ ]:
import os
import uuid
import requests
from typing import Annotated

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from agents import Agent, Runner, function_tool, AgentOutputSchema
from agents.memory import SQLiteSession

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
assert TAVILY_API_KEY, "Set TAVILY_API_KEY in your .env file"

## 3. Tavily Search Tool

In [ ]:
@function_tool
def tavily_search(
    query: Annotated[str, "The search query to send to Tavily"],
    max_results: Annotated[int, "Maximum number of results to return"] = 5,
) -> str:
    """Search the web using Tavily and return relevant results."""
    response = requests.post(
        "https://api.tavily.com/search",
        json={
            "api_key": TAVILY_API_KEY,
            "query": query,
            "max_results": max_results,
        },
        timeout=30,
    )
    response.raise_for_status()
    data = response.json()

    results = []
    for r in data.get("results", []):
        results.append(
            f"Title: {r['title']}\nURL: {r['url']}\nContent: {r['content']}\n"
        )
    return "\n---\n".join(results) if results else "No results found."

## 4. Pydantic Output Models

In [ ]:
class AnalystOutput(BaseModel):
    trends: list[str] = Field(description="Key trends identified from the research")
    risks: list[str] = Field(description="Potential risks or challenges")
    insights: list[str] = Field(description="Actionable insights and observations")


class FinalReport(BaseModel):
    executive_summary: str = Field(description="A concise executive summary (2-3 paragraphs)")
    markdown_report: str = Field(description="A detailed markdown-formatted report with sections and bullet points")
    follow_up_questions: list[str] = Field(description="3-5 follow-up questions for further research")

## 5. Researcher Agent (Swappable)

This function creates the Researcher agent. Replace it with a different builder to swap the researcher for another domain without changing the pipeline.

In [ ]:
def build_researcher_agent() -> Agent:
    """Build the default Researcher agent that uses Tavily search.

    Returns an Agent instance. Swap this function to plug in a
    different domain-specific researcher.
    """
    return Agent(
        name="Researcher",
        instructions=(
            "You are a thorough research assistant. "
            "When given a query, you MUST use the tavily_search tool to gather "
            "real sources from the web before summarizing. "
            "Make multiple searches if needed to cover different angles. "
            "Return a comprehensive research summary that includes key facts, "
            "data points, and source URLs."
        ),
        tools=[tavily_search],
    )

## 6. Analyst & Writer Agents

In [ ]:
analyst_agent = Agent(
    name="Analyst",
    instructions=(
        "You are a senior analyst. Given a research summary, identify the most "
        "important trends, potential risks, and actionable insights. "
        "Be specific and back up your analysis with evidence from the research."
    ),
    output_type=AgentOutputSchema(AnalystOutput, strict_json_schema=True),
)

writer_agent = Agent(
    name="Writer",
    instructions=(
        "You are an expert report writer. Given an analysis with trends, risks, "
        "and insights, produce a polished final report. "
        "The executive_summary should be 2-3 concise paragraphs. "
        "The markdown_report should be a detailed, well-structured document with "
        "headings, bullet points, and clear sections. "
        "Include 3-5 follow_up_questions that would deepen the research."
    ),
    output_type=AgentOutputSchema(FinalReport, strict_json_schema=True),
)

## 7. Pipeline

In [ ]:
async def run_pipeline(user_query: str, researcher_agent: Agent) -> FinalReport:
    """Orchestrate Researcher → Analyst → Writer.

    Args:
        user_query: The research question.
        researcher_agent: A pre-built Agent to use for the research step.

    Returns:
        FinalReport with executive_summary, markdown_report, and follow_up_questions.
    """
    session_id = f"pipeline-{uuid.uuid4().hex[:8]}"
    session = SQLiteSession(session_id)

    # Step 1 — Research
    print("[1/3] Researching...")
    research_result = await Runner.run(
        researcher_agent,
        input=user_query,
        session=session,
    )
    research_summary = research_result.final_output
    print(f"  ✔ Research done ({len(research_summary)} chars)")

    # Step 2 — Analysis
    print("[2/3] Analysing...")
    analyst_result = await Runner.run(
        analyst_agent,
        input=f"Analyse the following research:\n\n{research_summary}",
        session=session,
    )
    analysis: AnalystOutput = analyst_result.final_output
    print(f"  ✔ Analysis done — {len(analysis.trends)} trends, {len(analysis.risks)} risks, {len(analysis.insights)} insights")

    # Step 3 — Report
    print("[3/3] Writing report...")
    writer_input = (
        f"Trends:\n" + "\n".join(f"- {t}" for t in analysis.trends) + "\n\n"
        f"Risks:\n" + "\n".join(f"- {r}" for r in analysis.risks) + "\n\n"
        f"Insights:\n" + "\n".join(f"- {i}" for i in analysis.insights)
    )
    writer_result = await Runner.run(
        writer_agent,
        input=writer_input,
        session=session,
    )
    report: FinalReport = writer_result.final_output
    print("  ✔ Report ready\n")

    return report

## 8. Run Example

In [ ]:
from IPython.display import display, Markdown

researcher = build_researcher_agent()

report = await run_pipeline(
    user_query="What are the latest trends in AI agents and multi-agent systems in 2025?",
    researcher_agent=researcher,
)

### Executive Summary

In [ ]:
display(Markdown(report.executive_summary))

### Full Report

In [ ]:
display(Markdown(report.markdown_report))

### Follow-up Questions

In [ ]:
for i, q in enumerate(report.follow_up_questions, 1):
    display(Markdown(f"**{i}.** {q}"))